In [6]:
import numpy as np
import matplotlib.pyplot as plt
import time 
from collections import defaultdict
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity

plt.rcParams['font.sans-serif'] = ['Hiragino Sans GB', 'STHeiti', 'PingFang SC', 'Microsoft YaHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

np.random.seed(42)

第一步：生成模拟数据
我们创建一些简单的二维数据，方便可视化理解

In [ ]:
def generate_sample_data(n_samples=1000, dim=2):
    '''
    生成3个明显分离的高斯分布簇
    loc是均值mean
    scale是标准差
    '''
    cluster1 = np.random.normal(loc=[2,2], scale=0.5, size=(n_samples//3, dim))
    cluster2 = np.random.normal(loc=[8,3], scale=0.6, size=(n_samples//3, dim))
    cluster3 = np.random.normal(loc=[5,8], scale=0.4, size=(n_samples - 2*(n_samples//3), dim))

    # vertical stack 将多个数组沿第一个轴
    data = np.vstack([cluster1, cluster2, cluster3])
    return data

In [4]:
data = generate_sample_data() 
print(f'数据形状: {data.shape}')

数据形状: (1000, 2)


第二步：实现K-means聚类

这是IVF算法的核心预处理步骤

In [8]:
class SimpleKMeans:
    '''
    简单的K-Means算法实现
    '''
    def __init__(self, n_cluster=3, max_iters=100) -> None:
        self.n_cluster = n_cluster
        self.max_iters = max_iters
        self.centroids = None
        self.labels_ = None

    def fit(self, X):
        # num, dim
        n_samples, n_features = X.shape

        # 1.随机初始化质心
        random_indices = np.random.choice(n_samples, self.n_cluster, replace=False)
        self.centroids = X[random_indices]

        for it in range(self.max_iters):
            # 2. 分配每个点到最近的质心
            distances = euclidean_distances(X, self.centroids) # shape(n_sample, n_centroids)
            # 对每个数据点，找到距离最近的质心索引
            labels = np.argmin(distances, axis=1)   # shape(n_sample, 1)

            # 3. 更新质心位置
            # 对每个聚类 i（从 0 到 n_cluster-1）
            #   X[labels == i]：筛选出所有被分配到聚类 i 的数据点
            #   mean(axis=0)：计算这些点在每个特征维度上的均值，得到该聚类的新质心
            new_centroids = np.array([X[labels == i].mean(axis=0) for i in range(self.n_cluster)])
            # 如果所有对应质心都足够接近，返回 True，表示已收敛
            if np.allclose(self.centroids, new_centroids):
                # 如果质心不再显著变化，提前结束迭代，避免不必要的计算
                break
            self.centroids = new_centroids

        self.labels_ = labels
        return self

第三步：实现倒排文件索引（IVF）

In [ ]:
class SimpleIVF:
    '''
    简易IVF实现
    '''
    def __init__(self, n_clusters=3, n_probe=2) -> None:
        self.n_clusters = n_clusters
        self.n_probe = n_probe  # 搜索时探测的簇数量
        self.kmeans = None
        self.inverted_lists = None # 倒排列表
        self.centroids = None
        self.is_trained = False

    def train(self, data):
        ''' 对数据进行聚类 '''
        print('start training IVF index...')
        self.kmeans = SimpleKMeans(n_cluster=self.n_clusters)
        self.kmeans.fit(data)
        self.centroids = self.kmeans.centroids
        self.is_trained = True
        print(f'training completed, {self.centroids} clusters')

    def build_index(self, data):
        '''构建倒排索引'''
        if not self.is_trained:
            self.train(data)

        # 初始化倒排列表
        self.inverted_lists = defaultdict(list)

        # 将每个向量分配给最近的簇
        distances = euclidean_distances(data, self.centroids)
        labels = np.argmin(distances, axis=1) # vector_id -> cluster_id

        # 构建倒排列表：cluster_id -> 该Cluster的所有向量id
        for idx, label in enumerate(labels):
            self.inverted_lists[label].append(idx)

        print('inverted index build success.')
        for cluster_id, items in self.inverted_lists.items():
            print(f' cluster {cluster_id}: {len(items)} vectors')

    def search(self, query, k=5, data=None):
        '''IVF搜索，先找最近的cluster,然后在cluster内搜索'''
        if data is None:
            data = self.data

        # 1.粗略搜索：找到最近的n_probs个Cluster
        distances_to_centroids = euclidean_distances([query], self.centroids)[0]
        nearest_cluster_indices = np.argsort(distances_to_centroids)[:self.n_probe]

        # 2.精细搜索：在选中的cluster内进行暴力搜索
